def get_input_sizes(self, config):

        return [1, 1, 1, 1, 1, 1, 6, 6, 1, 1] 
        
for the values 6 is for R_arm, L_arm etc., 6 is for u_1:6, initial conditions for x, angular frequency, time  

We need

* Six individual parameters (R_arm, L_arm, etc.)

* Two arrays of size 6 (for u_1:6, initial conditions for x_1:6)

* One angular frequency (omega)

* One time (T)

In [83]:
import umbridge
import numpy as np

# Initialize the model
model = umbridge.HTTPModel("http://localhost:4242", "forward")

# Define the input parameters
R_arm = [0.5]     # Example value for R_arm
R_DC = [0.1]      # Example value for R_DC
R_AC = [0.2]      # Example value for R_AC
L_arm = [0.3]     # Example value for L_arm
L_DC = [0.4]      # Example value for L_DC
L_AC = [0.5]      # Example value for L_AC
u_parameters = [1, 2, 3, 4, 5, 6]  # Example values for u_1 to u_6 (size 6)
x_initial = [0, 0, 0, 0, 0, 0]      # Example initial conditions for x (size 6)
omega = [50]                  # Example angular frequency
resolution = 125e-6
T = [0.02]   
T_eval = np.arange(0, T[0]+resolution, resolution)                         # Example time


# Create the parameters list
parameters = [R_arm, R_DC, R_AC, L_arm, L_DC, L_AC, u_parameters, x_initial, omega, T]


# Call the model
result = model(parameters)


# Convert list into 1D array 
solution_x = np.array(result)
solution_x_1_6 = E.flatten()
print(solution_x_1_6)

[ 0.         -2.48783983 -5.99910785 -7.99881047 -5.54977257 -6.65972709]


The following code is used for solving the 2nd equation (3b in the paper), so that we get the solutions for $x_7,x_8,x_9,x_{10},x_{11},x_{12}$

In [84]:
from scipy.integrate import odeint

# T matrix 
matrix_T = np.array( [[1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                      [-1/np.sqrt(6), -1/np.sqrt(6), -1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                      [1/2, -1/2, 0, -1/2, 1/2, 0],
                      [1/np.sqrt(12), 1/np.sqrt(12), -1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)],
                      [-1/2, 1/2, 0, -1/2, 1/2, 0],
                      [-1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)],
                     ])

T_transpose = np.transpose(matrix_T)

deriv_x_7_12 = (T_transpose @ u_parameters)   *  (T_transpose @ solution_x_1_6)

def model(x, t):
    dx_dt = deriv_x_7_12  # The right-hand side of the equation is constant
    return dx_dt

# Initial condition
x0 = np.zeros(6)  # Assuming initial condition y(0) = [0, 0, 0, 0, 0, 0]
T = [0.02]
resolution = 125e-6
t_eval = np.arange(0, T[0]+resolution, resolution)

# Solve ODE
solution_x_7_12 = odeint(model, y0, t_eval)
print(solution_x_7_12)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.00379116e-04  1.53692877e-06  1.66904531e-04 -6.36291981e-03
  -7.47731615e-04 -8.29184472e-03]
 [-2.00758232e-04  3.07385755e-06  3.33809062e-04 -1.27258396e-02
  -1.49546323e-03 -1.65836894e-02]
 [-3.01137349e-04  4.61078632e-06  5.00713593e-04 -1.90887594e-02
  -2.24319484e-03 -2.48755342e-02]
 [-4.01516465e-04  6.14771510e-06  6.67618123e-04 -2.54516792e-02
  -2.99092646e-03 -3.31673789e-02]
 [-5.01895581e-04  7.68464387e-06  8.34522654e-04 -3.18145991e-02
  -3.73865807e-03 -4.14592236e-02]
 [-6.02274697e-04  9.22157265e-06  1.00142719e-03 -3.81775189e-02
  -4.48638969e-03 -4.97510683e-02]
 [-7.02653813e-04  1.07585014e-05  1.16833172e-03 -4.45404387e-02
  -5.23412130e-03 -5.80429131e-02]
 [-8.03032930e-04  1.22954302e-05  1.33523625e-03 -5.09033585e-02
  -5.98185292e-03 -6.63347578e-02]
 [-9.03412046e-04  1.38323590e-05  1.50214078e-03 -5.72662783e-02
  -6.72958453e-03 -7.4626

This is for the last equation $\tilde{y} = \tilde{C}x_{1:6}$. To find the initial conditions for $x_{1:6}$, we are using the function of $y_ref$ at time $t=0$. Then we have the following linear system 

\begin{equation}
\left(\begin{array}{cc} 
y_{1,ref}(0) \\
y_{2,ref}(0) \\
y_{3,ref}(0) \\
\end{array}\right)
= - \left(\begin{array}{c} 
1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 \\
\end{array}\right) 
\left(\begin{array}{c} 
x_1(0) \\
x_2(0) \\
x_3(0) \\
x_4(0) \\
x_5(0) \\
x_6(0) \\
\end{array}\right)  
\end{equation}

Since we dont have square matrix, the equation is under or over determined. By using the least square method, we can estimate the solution.

In [73]:
y_ref = [0] * 3
I_AC = 15
phi_AC = 0
matrix_C = np.array([[1, 0, 0, 1, 0, 0],
              [0, 1, 0, 0, 1, 0],
              [0, 0, 1, 0, 0, 1]])

#print(np.linalg.inv(matrix_C))


def y(T, omega):
        for k in range(1, 4):
            y_ref[k-1]= I_AC * np.cos (omega * T - (2 * np.pi * (k-1))/3 -phi_AC)

        return y_ref

y0_values = y(0, 50)
x0, residuals, rank, s = np.linalg.lstsq(matrix_C, y0_values, rcond=None)
print("initial condition values using least squares:", x0)

initial condition values using least squares: [ 7.5  -3.75 -3.75  7.5  -3.75 -3.75]


In [79]:
T = [0.02]
resolution = 125e-6
t_eval = np.arange(0, T[0]+resolution, resolution)
print(t_eval)

[0.       0.000125 0.00025  0.000375 0.0005   0.000625 0.00075  0.000875
 0.001    0.001125 0.00125  0.001375 0.0015   0.001625 0.00175  0.001875
 0.002    0.002125 0.00225  0.002375 0.0025   0.002625 0.00275  0.002875
 0.003    0.003125 0.00325  0.003375 0.0035   0.003625 0.00375  0.003875
 0.004    0.004125 0.00425  0.004375 0.0045   0.004625 0.00475  0.004875
 0.005    0.005125 0.00525  0.005375 0.0055   0.005625 0.00575  0.005875
 0.006    0.006125 0.00625  0.006375 0.0065   0.006625 0.00675  0.006875
 0.007    0.007125 0.00725  0.007375 0.0075   0.007625 0.00775  0.007875
 0.008    0.008125 0.00825  0.008375 0.0085   0.008625 0.00875  0.008875
 0.009    0.009125 0.00925  0.009375 0.0095   0.009625 0.00975  0.009875
 0.01     0.010125 0.01025  0.010375 0.0105   0.010625 0.01075  0.010875
 0.011    0.011125 0.01125  0.011375 0.0115   0.011625 0.01175  0.011875
 0.012    0.012125 0.01225  0.012375 0.0125   0.012625 0.01275  0.012875
 0.013    0.013125 0.01325  0.013375 0.0135   0.013